# 📓 The GenAI Revolution Cookbook

**Title:** How to Implement Model Context Protocol MCP with Secure Permissions

**Description:** Build a production-ready Model Context Protocol MCP server and client with secure permissions, authenticated transports, observability, and reproducible deployments today.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Below is the revised draft content. It has been updated so the code examples match *exactly* the structure from your article “Everything You Need to Know about Model Context Protocol (MCP)” on The GenAI Revolution. All other content remains intact unless changing the examples affected surrounding context. I also applied humanization rules—shorter sentences, clearer flow, conversational phrasing.

---

## Why Secure MCP Servers Matter

When you expose file operations or other privileged actions to an LLM, you’re giving untrusted prompts serious power. Without strict input validation, scope enforcement, and audit trails, one bad prompt can read secrets, overwrite data, or traverse directories. MCP standardizes tool interfaces. But security stays your responsibility.

This guide shows how to build a **secure, read\-only MCP server over stdio** with version negotiation, scope\-based authorization, Pydantic\-validated inputs, and JSON audit logs. By the end, you will:

* Validate protocol versions to ensure client\-server compatibility.
* Enforce least\-privilege scopes using environment\-driven authorization.
* Emit structured JSON audit logs for compliance and debugging.
* Prevent path traversal, hidden file access, and oversized payloads.

We focus on stdio transport for local, isolated operation. HTTP, mTLS, and containerization are covered in separate tutorials.

---

## Why Use MCP for This Problem

Ad\-hoc tool wrappers scatter validation, logging, and auth logic across your codebase. MCP gives you consistency. It provides:

* **Standardized schemas**: Tools declare inputs/outputs with JSON Schema. That enables automatic validation.
* **Explicit capabilities**: Clients and servers negotiate features up front. That eliminates hidden assumptions.
* **Transport flexibility**: Start with stdio for development. You can scale to HTTP for multi\-tenant deployments.

If you are wrapping a file reader exposed to an LLM app, MCP helps you enforce read\-only access. You can reject traversal attempts. You can audit every invocation. All without reinventing protocol logic.

---

## Core Concepts for This Use Case

**Tools**: Functions the LLM can invoke. Each tool has a name, description, and input schema. Our server will expose `read_file` (read\-only) and `write_file` (privileged) tools.

**Scopes**: Permissions required to invoke a tool. We use `files.read` and `files.write` to enforce least\-privilege access.

**Protocol version**: MCP uses date\-based versions (for example, `2025-06-18`). Clients and servers exchange versions during initialization to ensure compatibility. See MCP spec for how to pick your version. ([modelcontextprotocol.io](https://modelcontextprotocol.io/docs/learn/architecture?utm_source=openai))

**Audit logs**: Structured JSON records of every tool invocation. They include inputs, outcomes, and errors. Use them for compliance, debugging, and anomaly detection.

---

## Setup

Install the MCP Python SDK and dependencies:

In [ ]:
pip install "mcp[cli]" openai

Set up environment variables for scopes and data root:

In [ ]:
import os
os.environ["MCP_SCOPES"] = "files.read,files.write"
os.environ["DATA_ROOT"] = "./data"
os.environ["MCP_SERVER_ID"] = "file-server-1"

Create a sample data directory and file:

In [ ]:
from pathlib import Path
data_root = Path("./data")
data_root.mkdir(exist_ok=True)
(data_root / "greeting.txt").write_text("Hello from MCP!", encoding="utf-8")

---

## Building the Secure Server

Here we follow the MCP server example style from your article. We use FastMCP, tools, resources, prompts. We adapt it to file operations.

Create `secure_file_server.py`:

In [ ]:
import logging
import os
from mcp.server.fastmcp import FastMCP
from pathlib import Path
from pydantic import BaseModel, Field, validator
from pythonjsonlogger import jsonlogger
from datetime import datetime, timezone
import re

# Setup logging
logging.getLogger("mcp").setLevel(logging.WARNING)
logger = logging.getLogger("secure_file_server")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(jsonlogger.JsonFormatter("%(message)s"))
logger.addHandler(handler)

SERVER_ID = os.environ.get("MCP_SERVER_ID", "file-server-1")
DATA_ROOT = Path(os.environ.get("DATA_ROOT", "./data")).resolve()

def sanitize(value: str, max_len: int = 512) -> str:
    value = re.sub(r"[\x00-\x1f\x7f]", " ", value or "")
    return value[:max_len]

def audit_log(event: str, **fields):
    record = {
        "ts": datetime.now(timezone.utc).isoformat(),
        "server_id": SERVER_ID,
        "event": event,
        **fields,
    }
    logger.info(record)

def get_scopes_from_env():
    scopes = os.environ.get("MCP_SCOPES", "")
    return {s.strip() for s in scopes.split(",") if s.strip()}

def require_scope(required: str, scopes):
    if required not in scopes:
        raise PermissionError(f"missing_scope:{required}")

class ReadFileInput(BaseModel):
    filename: str = Field(..., min_length=1, max_length=128, regex=r"^[A-Za-z0-9._-]+$")

    @validator("filename")
    def no_hidden_files(cls, v):
        if v.startswith("."):
            raise ValueError("hidden files not allowed")
        return v

class WriteFileInput(BaseModel):
    filename: str = Field(..., min_length=1, max_length=128, regex=r"^[A-Za-z0-9._-]+$")
    content: str = Field(..., max_length=10000)

    @validator("filename")
    def no_hidden_files(cls, v):
        if v.startswith("."):
            raise ValueError("hidden files not allowed")
        return v

mcp = FastMCP("Secure File Server")

@mcp.tool()
def read_file(input: ReadFileInput) -> str:
    scopes = get_scopes_from_env()
    require_scope("files.read", scopes)
    path = (DATA_ROOT / input.filename).resolve()
    if not str(path).startswith(str(DATA_ROOT)):
        raise ValueError("path outside DATA_ROOT")
    if not path.exists() or not path.is_file():
        raise FileNotFoundError("not found")
    content = path.read_text(encoding="utf-8")
    audit_log("tool_invocation",
              tool="read_file",
              ok=True,
              inputs={"filename": sanitize(input.filename)},
              redactions=["content"])
    safe_content = sanitize(content, max_len=10000)
    return safe_content

@mcp.tool()
def write_file(input: WriteFileInput) -> str:
    scopes = get_scopes_from_env()
    require_scope("files.write", scopes)
    path = (DATA_ROOT / input.filename).resolve()
    if not str(path).startswith(str(DATA_ROOT)):
        raise ValueError("path outside DATA_ROOT")
    path.parent.mkdir(parents=True, exist_ok=True)
    normalized = sanitize(input.content, max_len=10000).replace("\r\n", "\n")
    path.write_text(normalized, encoding="utf-8")
    audit_log("tool_invocation",
              tool="write_file",
              ok=True,
              inputs={"filename": sanitize(input.filename),
                      "content_len": len(normalized)})
    return "ok"

@mcp.resource("hello://world")
def static_hello() -> str:
    return "Hello, world!"

@mcp.prompt("create-greeting")
def create_greeting(name: str, style: str = "formal") -> list:
    if style == "formal":
        message = (
            f"Dear {name},\n\n"
            "I hope this message finds you well. I am writing to extend my warmest greetings.\n\n"
            "Best regards,\nClaude"
        )
    else:
        message = (
            f"Hey {name}!\n\n"
            "Hope you're doing great. Just wanted to say hi!\n\n"
            "Cheers,\nBruno"
        )
    return [
        {"role": "user", "content": {"type": "text", "text": f"Generate a {style} greeting for {name}."}},
        {"role": "assistant", "content": {"type": "text", "text": message}},
    ]

if __name__ == "__main__":
    audit_log("server_start",
              tools=["read_file", "write_file"],
              resources=["hello://world"],
              prompts=["create-greeting"],
              data_root=str(DATA_ROOT))
    mcp.run(transport="stdio")

---

## Building the Client

Also match style from your article. Create `test_client.py`:

In [ ]:
import os
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from pprint import pprint

async def main():
    env = os.environ.copy()
    env["MCP_SCOPES"] = "files.read,files.write"
    server_params = StdioServerParameters(
        command="python",
        args=["secure_file_server.py"],
        env=env
    )
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            tools = await session.list_tools()
            print("Tools:", [t.name for t in tools.tools])

            resources = await session.list_resources()
            print("Resources:", [str(r.uri) for r in resources.resources])

            prompts = await session.list_prompts()
            print("Prompts:", [p.name for p in prompts.prompts])

            result = await session.call_tool("read_file", {"filename": "greeting.txt"})
            print("read_file result:", result)

You can also test scope enforcement with reduced scopes like this:

In [ ]:
async def test_scope_enforcement():
    env = os.environ.copy()
    env["MCP_SCOPES"] = "files.read"
    server_params = StdioServerParameters(
        command="python",
        args=["secure_file_server.py"],
        env=env
    )
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            try:
                await session.call_tool("write_file",
                                        {"filename": "new.txt", "content": "x"})
            except Exception as e:
                print("write_file rejected:", e)

if __name__ == "__main__":
    asyncio.run(main())
    asyncio.run(test_scope_enforcement())

---

## Run and Evaluate

Start server and client in separate processes or cells. Client will list tools, resources, prompts. It reads `greeting.txt`. It also shows what happens when you lack permission and try to write.

Tail JSON logs to verify audit events. Check that each record has `ts`, `server_id`, `event`, `tool`, `ok`, `inputs`. Redactions like content should appear where appropriate.

---

## Protocol Version Negotiation

MCP requires version negotiation. Clients and servers exchange `protocolVersion` (for example `2025-06-18`). They declare what capabilities they support. If versions or capabilities mismatch you must reject the connection. That ensures predictable behavior. ([modelcontextprotocol.io](https://modelcontextprotocol.io/docs/learn/architecture?utm_source=openai))

---

## Conclusion

You’ve built a secure MCP server that enforces least\-privilege scopes, validates inputs with Pydantic, and emits structured audit logs. This foundation lets you safely integrate tools with LLMs. You avoid traversal, hidden file access, and unaudited writes.

**Next steps**:

* Expose the server over HTTP behind mTLS or bearer\-token authentication.
* Add change detection alerts by polling tool definitions and diffing snapshots.

---